# Block Parsing - BX Exercises

<hr style="border: 0.5px dashed #000;">

## 1. Serialize the genesis block header

### Request block header information from the genesis block

In [ ]:
# bx fetch-header --height [height]
bx fetch-header --height 0


### Transform all header parts into serialised hex form (Part 1)

* `Bits (from dec to hexle)`
* `nonce (from dec to hexle)`
* `Timestamp (from dec to hexle)`

In [ ]:
# Use sfk tool:
# sfk num [decimal number] -show hexle

echo "Bits:"
sfk num 486604799 -show hexle

echo "Nonce:"
sfk num 2083236893 -show hexle

echo "Timestamp:"
sfk num 1231006505 -show hexle


### Transform all header parts into serialised hex form (Part 2)

* `Merkle root (to little endian)`
* `Previous block hash (to little endian)`


* [Online hex endianess reversal tool](https://onlinehextools.com/reverse-hex-digits)

In [ ]:
# https://onlinehextools.com/reverse-hex-digits
# Or ...
a=18dd438bb9571d2f7be12a8e58bc06efb84f4c6f7c914e5b805e9994766c84bf
b=982051fd1e4ba744bbbe680e1fee14677ba1a3c3540bf7b1cdb606e857233e0e
echo ${a:64:2}${a:62:2}${a:60:2}${a:58:2}${a:56:2}${a:54:2}${a:52:2}${a:50:2}${a:48:2}${a:46:2}${a:44:2}${a:42:2}${a:40:2}${a:38:2}${a:36:2}${a:34:2}${a:32:2}${a:30:2}${a:28:2}${a:26:2}${a:24:2}${a:22:2}${a:20:2}${a:18:2}${a:16:2}${a:14:2}${a:12:2}${a:10:2}${a:8:2}${a:6:2}${a:4:2}${a:2:2}${a:0:2}
echo ${b:64:2}${b:62:2}${b:60:2}${b:58:2}${b:56:2}${b:54:2}${b:52:2}${b:50:2}${b:48:2}${b:46:2}${b:44:2}${b:42:2}${b:40:2}${b:38:2}${b:36:2}${b:34:2}${b:32:2}${b:30:2}${b:28:2}${b:26:2}${b:24:2}${b:22:2}${b:20:2}${b:18:2}${b:16:2}${b:14:2}${b:12:2}${b:10:2}${b:8:2}${b:6:2}${b:4:2}${b:2:2}${b:0:2}


### Concatenate all parts of the header to final form.

In [ ]:
# var1= ...
# var2= ...
# var3= ...
# echo $var0$var1$var1...

version=01000000
prev_hash=d61dcc0e13256adb725861971b4bae753859ee61e406392305a4fddc54b55721
merkle_root=3ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e5e4a
timestamp=29AB5F49
bits=FFFF001D
nonce=1DAC2B7C

echo $version$prev_hash$merkle_root$timestamp$bits$nonce 

## 2. Verify the genesis header is indeed organised in the blockchain

* Where is the header committed to?
* What form does this commitment have?

In [ ]:
# Show header commitment location.
bx fetch-header --height 1


In [ ]:
# Derive header commitment from genesis header(80B)
bx bitcoin256 0100000000000000000000000000000000000000000000000000000000000000000000003ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e5e4a29AB5F49FFFF001D1DAC2B7C


## 3. Demonstrate the block height commitment.

* Coinbase TXID at blockheight 548'837: `ebeaf0c4be2fdea6b759b233c895d7cad0856a642fac44d4f7e67ad8d1e1e5b5`
* Coinbase TXID at blockheight 548'823: `0bda48fc47764b24a15a4d0a796243447526764f9d50bcbd2df268ecd054d998`

In [ ]:
# Get coinbase transaction.
bx fetch-tx ebeaf0c4be2fdea6b759b233c895d7cad0856a642fac44d4f7e67ad8d1e1e5b5


In [ ]:
# Remember to check Endianness.
# Convert value to decimal:
# sfk dec [hex]

sfk dec 085fe5


### Try converting the remainder of the input script into ascii/utf8
* [Webtool](https://sites.google.com/site/nathanlexwww/tools/utf8-convert) for hex to utf-8 conversion

In [ ]:
# echo [script] | bx script-encode
echo "[0cfddf5b] ver [4254432e434f4d2ffabe6d6d7774fea0d063067716a4e4565cb351f56ebd1e0da7f79716d9c3c703790dd326010000]" \
| bx script-encode 


* Miner text at height 548'837: `ퟛb/BTC.COM/꾭mwtcw䤖\㑵n힍緗كǃy`...
* Miner text at height 548'823:  `ן/ViaBTC/Mined by 13910851095/,꾭m㚩h#Ϙ8kH㗡DMy㣞˓ݔ㖐ۈռd숍⥥褍룒A`...

## 4. Determine fees and subsidy of block.

* At height `422108`, Coinbase TXID: `bf846c7694995e805b4e917c6f4c4fb8ef06bc588e2ae17b2f1d57b98b43dd18`
* At height `312312`
* At height `5124`

In [ ]:
# Halvenings at given height
expr 422108 / 210000


In [ ]:
# Subsidy at given height
expr 5000000000 / 2 / 2


In [ ]:
# Compute fees from coinbase tx output amount.
bx fetch-tx bf846c7694995e805b4e917c6f4c4fb8ef06bc588e2ae17b2f1d57b98b43dd18 -f json \
| jq ".transaction.outputs[0].value"


In [ ]:
expr 1266815487 - 1250000000


## 4. POW: How many attempts are required for a valid header hash?

* For the following heights:
    * Genesis block
    * Block at height `10000`
    * Block at height `500000`
* [Big hex/dec online calculator](https://defuse.ca/big-number-calculator.htm)

In [ ]:
# Get bits from block at height.
bx fetch-header --height 500000


In [ ]:
# Convert decimal difficulty into hex
bits_decimal=402691653
bits_hex_le=$(sfk num $bits_decimal -show hexle)

# Derive target.
coefficient=${bits_hex_le:0:6}
exponent_decimal=$(sfk dec ${bits_hex_le:6:2})
hex_zeros=$(expr $exponent_decimal - 3)
leading_null_bytes=$(expr 32 - 3 - $hex_zeros)

# Print out target.
for ((i=1;i<=$leading_null_bytes;i++)); do 
    printf "00"
done
printf $coefficient
for ((i=1;i<=$hex_zeros;i++)); do 
    printf "00"
done

In [ ]:
# Compute attempts necessary to hash under target.
# 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF/0X0000000000000000459600000000000000000000000000000000000000000000
# Via online calculator: 67863804850939102655


### How long would it take the following hardware to mine block at height 500k?
* Intel(R) Core(TM) i3 CPU, M 350 @ 2.27GHz 
    * `1486742 (H/Sec)`

* Intel(R) Xeon(R) CPU , E5530 @ 2.40GHz
    * `7138070 (H/Sec)`

* Bitmain Antminer S9 Bitcoin Miner
    * `13.5 (TH/Sec)`  
    
* [Big hex/dec online calculator](https://defuse.ca/big-number-calculator.htm)

In [ ]:
# Antminer S9:
# Via online calculator: 67863804850939102655 / 13500000000000 / 3600 /24 = 58 Days
